# Petfinder.my - Pawpularity Contest
Predict the popularity of shelter pet photos
<img src="https://storage.googleapis.com/kaggle-competitions/kaggle/25383/logos/header.png"></img>

Analyze raw images and metadata to predict the “Pawpularity” of pet photos. The Pawpularity Score is derived from each pet profile's page view statistics at the listing pages, using an algorithm that normalizes the traffic data across different pages, platforms (web & mobile) and various metrics.

## Photo Metadata
The train.csv and test.csv files contain metadata for photos in the training set and test set, respectively. Each pet photo is labeled with the value of 1 (Yes) or 0 (No) for each of the following features:

- Focus - Pet stands out against uncluttered background, not too close / far.
- Eyes - Both eyes are facing front or near-front, with at least 1 eye / pupil decently clear.
- Face - Decently clear face, facing front or near-front.
- Near - Single pet taking up significant portion of photo (roughly over 50% of photo width or height).
- Action - Pet in the middle of an action (e.g., jumping).
- Accessory - Accompanying physical or digital accessory / prop (i.e. toy, digital sticker), excluding collar and leash.
- Group - More than 1 pet in the photo.
- Collage - Digitally-retouched photo (i.e. with digital photo frame, combination of multiple photos).
- Human - Human in the photo.
- Occlusion - Specific undesirable objects blocking part of the pet (i.e. human, cage or fence). Note that not all blocking objects are considered occlusion.
- Info - Custom-added text or labels (i.e. pet name, description).
- Blur - Noticeably out of focus or noisy, especially for the pet’s eyes and face. For Blur entries, “Eyes” column is always set to 0.

# Importing Libraries

In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import warnings 
import multiprocessing

warnings.filterwarnings('ignore')

In [3]:
TRAIN_PATH = "../input/petfinder-pawpularity-score/train/"
TEST_PATH = "../input/petfinder-pawpularity-score/test/"
ROOT_LOGDIR = os.path.join(os.curdir, "logs")

IMAGE_SIZE = 224
BATCH_SIZE = 30
NO_OF_EPOCHS = 100

input_cols = ['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
              'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur']

In [9]:
train_df = pd.read_csv("../input/petfinder-pawpularity-score/train.csv")
test_df = pd.read_csv("../input/petfinder-pawpularity-score/test.csv")

# Adding paths for images
train_df["Path"] = TRAIN_PATH + train_df["Id"] + ".jpg"
test_df["Path"] = TEST_PATH + test_df["Id"] + ".jpg"

# Normalize the labels
scaler = StandardScaler()
train_df["Pawpularity"] = scaler.fit_transform(train_df[["Pawpularity"]])
train_df.head()

In [37]:
test_df.head()

In [10]:
print("Training set instances:", train_df.shape[0])
print("Testing set instances:", test_df.shape[0])

# Creating keras Dataset

In [11]:
def generate_image_dataset(input_data, label):
    filepath, tabular_cols = input_data
    image = tf.io.read_file(filepath)
    image = tf.image.decode_jpeg(image, channels=3) 
    iamge = tf.cast(image, tf.float32) / 255.0
    image = tf.image.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
    return (image, tabular_cols), label


train, valid = train_test_split(train_df, test_size=0.1)
dataset_train = (tf.data.Dataset.from_tensor_slices(((train["Path"].values, train[input_cols].values),
                                                    train["Pawpularity"].values )).
                 map(generate_image_dataset).batch(BATCH_SIZE).prefetch(1))
dataset_valid = (tf.data.Dataset.from_tensor_slices(((valid["Path"].values, valid[input_cols].values),
                                                    valid["Pawpularity"].values)).
                map(generate_image_dataset).batch(BATCH_SIZE).prefetch(1))

In [12]:
data_augmentation = keras.Sequential([
        keras.layers.experimental.preprocessing.Normalization(),
        keras.layers.experimental.preprocessing.Resizing(IMAGE_SIZE, IMAGE_SIZE),
        keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
        # random rotation in range [-20% * 2pi, 20% * 2pi]
        keras.layers.experimental.preprocessing.RandomRotation(factor=0.02), 
        # A positive value means zooming out, while a negative value means zooming in.
        keras.layers.experimental.preprocessing.RandomZoom(
            height_factor=0.2, width_factor=0.2
        # output zoomed out vertically in range [20%, 20%]
        # output zoomed out horizontally in the range [20%, 20%]
        )
    ],
    name="data_augmentaion"
)

In [13]:
def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(ROOT_LOGDIR, run_id)

In [24]:
def train_experimental_model(model, optimizer,
                             checkpoint_name=None):
    model.compile(
        optimizer=optimizer,
        loss=keras.losses.MeanSquaredError(),
        metrics=[
            keras.metrics.RootMeanSquaredError(name="rmse"),
            "mae",
        ]
    )
    
    # Create callbacks
    if checkpoint_name==None:
        raise ValueError("Provide checpoint model name.")
    checkpoint_callback = keras.callbacks.ModelCheckpoint(
        checkpoint_name,
        monitor="val_rmse",
        save_best_only=True,
        save_weights_only=False
    )
    early_stopping_callback = keras.callbacks.EarlyStopping(
        patience=10,
        restore_best_weights=True,
        min_delta=1e-4,
    )
    reduce_lr = keras.callbacks.ReduceLROnPlateau(
        monitor="val_rmse",
        factor=0.3,
        patience=3,
        min_lr=1e-7
    )
    run_logdir = get_run_logdir()
    tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
    
    history = model.fit(
        dataset_train,
        validation_data=dataset_valid,
        batch_size=BATCH_SIZE,
        epochs=NO_OF_EPOCHS,
        callbacks=[checkpoint_callback, early_stopping_callback, reduce_lr, tensorboard_cb]
    )
    return history

# Let's train a EfficientNet CNN

In [25]:
inputs_image = keras.layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
inputs_tabular = keras.layers.Input(shape=(len(input_cols)))

tabular_dense = keras.layers.Dense(16, activation="relu")(inputs_tabular)
tabular_dense = keras.layers.Dropout(rate=0.4)(tabular_dense)
tabular_dense = keras.layers.Dense(16, activation="relu")(tabular_dense)
tabular_dense = keras.layers.Dropout(rate=0.4)(tabular_dense)
tabular_dense = keras.layers.Dense(16, activation="relu", 
                                   kernel_regularizer=keras.regularizers.l2(0.01))(tabular_dense)
tabular_dense = keras.layers.Dropout(rate=0.4)(tabular_dense)

augmented_image = data_augmentation(inputs_image)
efficient_net = keras.applications.EfficientNetB1(
    include_top=False,
    weights=None,
    input_tensor=augmented_image,
    input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3),
    pooling=None,
    classes=None,
    classifier_activation=None
)
efficient_net_output = efficient_net(augmented_image)
globalAvgPool2D = keras.layers.GlobalAvgPool2D()(efficient_net_output)
dense_layer_1 = keras.layers.Dense(100, activation="relu")(globalAvgPool2D)
dense_layer_1 = keras.layers.Dropout(rate=0.4)(dense_layer_1)
dense_layer_2 = keras.layers.Dense(30, activation="relu")(dense_layer_1)
dense_layer_2 = keras.layers.Dropout(rate=0.4)(dense_layer_2)

concatenate = keras.layers.Concatenate(axis=1)([dense_layer_2, tabular_dense])
dense_layer_out = keras.layers.Dense(1, 
                                     kernel_regularizer=keras.regularizers.l2(0.01))(concatenate)

efficientNet_model = keras.Model(inputs=(inputs_image, inputs_tabular), outputs=dense_layer_out)

In [26]:
efficientNet_model.summary()

In [30]:
history = train_experimental_model(efficientNet_model, optimizer=keras.optimizers.Adam(),
                                   checkpoint_name="efficientNetB1_model.h5")

# Visualization on Tensorboard

In [29]:
# TODO: Comment/Delete this Before commiting
# import os
# import multiprocessing

# # !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
# # !unzip ngrok-stable-linux-amd64.zip

# pool = multiprocessing.Pool(processes = 10)
# results_of_processes = [pool.apply_async(os.system, args=(cmd, ), callback = None )
#                         for cmd in [
#                         f"tensorboard --logdir ./logs/ --host 0.0.0.0 --port 6006 &",
#                         "./ngrok http 6006 &"
#                         ]]

# ! curl -s http://localhost:4040/api/tunnels | python3 -c \
#     "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

# Submission 

In [40]:
def generate_test_dataset(filepath, tabular_cols):
    image = tf.io.read_file(filepath)
    image = tf.image.decode_jpeg(image, channels=3) 
    iamge = tf.cast(image, tf.float32) / 255.0
    image = tf.image.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
    return (image, tabular_cols), 0


def generate_submission_csv(model):
    dataset_test = (tf.data.Dataset.from_tensor_slices((test_df["Path"].values, test_df[input_cols].values)).
                    map(generate_test_dataset).batch(BATCH_SIZE).prefetch(1))
    submission_df = test_df[["Id"]]
    submission_df = submission_df.assign(Pawpularity=scaler.inverse_transform(model.predict(dataset_test).reshape(-1)))
    submission_df.to_csv("submission.csv", index=False)
    
generate_submission_csv(efficientNet_model)